# Training Potts Models with Contrastive Divergence for Protein Design

This notebook contains the "Nice to haves" for the project.

In [1]:
!git clone https://github.com/igemto-drylab/CSBERG-ML.git
%cd CSBERG-ML
from util import *
import tqdm.notebook as tq
%cd ..

fatal: destination path 'CSBERG-ML' already exists and is not an empty directory.
/Users/syedather/Desktop/research/griffiths/potts/notebooks/CSBERG-ML
/Users/syedather/Desktop/research/griffiths/potts/notebooks


In [2]:
plt.rcParams['figure.dpi'] = 350

In [3]:
!git clone https://github.com/ranganathanlab/bmDCA.git

fatal: destination path 'bmDCA' already exists and is not an empty directory.


In [ ]:
!sudo apt-get update
!sudo apt-get install git gcc g++ automake autoconf pkg-config \
  libarmadillo-dev libopenblas-dev libarpack++2-dev

Password:

In [ ]:
%cd bmDCA
!bash autogen.sh --prefix=/usr/local && \
%cd ..

In [ ]:
%%shell
cd bmDCA
make -j4 && \
make install
cd ..

In [ ]:
!mkdir results

In [ ]:
%cd /content

In [ ]:
def read_fasta(fname, onehot=True):
    seqs = []
    seq_list = []
    s = ""
    with open(fname) as f:
        line = f.readline()
        while line:
            if line.startswith(">"):
                if s != "":
                    seqs.append(s)
                s = ""
            elif len(line) > 0:
                s += line.strip()
            line = f.readline()
        seqs.append(s)
        seq_list.append(list(s))
    if onehot:
        return get_X(seqs)
    else:
        return np.array(seqs)

def read_fasta_seqs(fname):
    seqs = []
    s = ""
    with open(fname) as f:
        line = f.readline()
        while line:
            if line.startswith(">"):
                if s != "":
                    seqs.append(list(s))
                s = ""
            elif len(line) > 0:
                s += line.strip()
            line = f.readline()
        seqs.append(list(s))
    return np.array(seqs)

In [ ]:
def trim_seqs(seqs, save_name):
    """
    seqs: sequences (list of letters not one-hot encoded)
    save_name: output fasta filename

    """
    mask = np.zeros(len(seqs[0]), dtype=np.bool)
    for i in range(len(seqs[0])):
        gaps = 0
        for s in seqs:
            if s[i] == '-':
                gaps += 1
        if gaps/len(seqs) < 0.67:   # keep positions where less than 2/3rd are gaps
            mask[i] = True
    seqs = seqs[:,mask]

    towrite = ""
    for i in range(len(seqs)):
        if sum(seqs[i][100:] == "-") < 50:  # don't take sequences with many gaps
            towrite += ">{}\n".format(i)
            seq = "".join(seqs[i][:]) + "\n"
            towrite += seq.upper()   
    with open(save_name,'w') as f:
        f.write(towrite)

In [ ]:
seqs = read_fasta_seqs("/content/lcc_pfam.txt")
trim_seqs(seqs, "lcc_short.fasta")